In [2]:
import pandas as pd
import numpy as np
import folium
import geopandas as gp


## Pull out the suppresed trips and assign them to census tracts

* Trips are suppressed to community areas when there is less than 3 trips for OD-15min 
* Assign those trips randomly to the census tracts within the community area that the suppressed trip is assigned to

In [6]:
tods = [1,2,3,4,5]
years = [2018,2019,2020]


In [7]:
agg = {'SCALED_SUP_PRIVATE_TRIPS':'sum', 'SCALED_SUP_SHARED_TRIPS':'sum', 'Pickup Community Area':'first', 'Dropoff Community Area':'first'}

In [30]:
df2 = pd.read_hdf('D:/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Chicago_TNC_Trips_20.H5', where = 'YEAR == ' + str(year), key = 'Weekday_' + str(tod))


In [31]:
df2

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,YEAR,MONTH,DOW,HOUR


In [11]:
store = pd.HDFStore('D:/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Incomplete H5 file/Chicago_TNC_Trips_20.H5')


In [12]:
#df = pd.read_hdf('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Chicago_TNC_Trips_20.H5', where = ['YEAR == 2019'], key = 'Weekday_' + str(1))

In [28]:
df = store.select(where = ['YEAR == 2018'], key = 'Weekday_' + str(1))

In [29]:
df[df['MONTH'] == 11]['Trip Start Timestamp'].min()

Timestamp('2018-11-01 01:00:00')

In [ ]:
conflation = pd.read_csv('D:/TNC-Demand-Model/Inputs/Chicago Community Areas/Community_Area_to_Census_Tract.csv')
conflation['COMMUNITY_AREA'] = conflation.area_num_1.astype(float)
store = pd.HDFStore('D:/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Incomplete H5 file/Chicago_TNC_Trips_20.H5')

for year in years:  
    if year == 2018:
        months = [11,12]
    elif year == 2020:
        months = [1,2]
    else:
        months = [1,2,3,4,5,6,7,8,9,10,11,12]
        
    print('Working on year ' + str(year))
    df5 = pd.DataFrame()
    
    for month in months:
        print('Working on month ' + str(month))
        df4 = pd.DataFrame()
        
        for tod in tods:
            print('Working on tod ' + str(tod))

            df3 = pd.DataFrame()
            df2 = pd.DataFrame()
            df = pd.DataFrame()
            df = store.select(where = ['YEAR == ' + str(year)], key = 'Weekday_' + str(tod))

            print('Filtering the Data!')
           # df = df[df['MONTH'].isin(months)]
            df['DAY'] = df['Trip Start Timestamp'].dt.day

            print(str(len(df[(~np.isnan(df['Pickup Community Area']))&(np.isnan(df['Pickup Census Tract']))&(~np.isnan(df['Dropoff Community Area'])) & (np.isnan(df['Dropoff Census Tract']))])) +  ' Trip Records with Suppressed Origin and Destination out of ' + str(len(df)) )
            print(str(len(df[((np.isnan(df['Pickup Community Area']))&(~np.isnan(df['Pickup Census Tract'])))|((np.isnan(df['Dropoff Community Area'])) & (np.isnan(df['Dropoff Census Tract'])))])) +  ' Trip Records Outside of Chicago but within cook county out of ' + str(len(df)) )
            print(str(len(df[((~np.isnan(df['Pickup Community Area']))&(~np.isnan(df['Pickup Census Tract'])))&((~np.isnan(df['Dropoff Community Area'])) & (~np.isnan(df['Dropoff Census Tract'])))])) +  ' Trip Records with both trip ends within Chicago out of ' + str(len(df)) )
            print(str(len(df[(np.isnan(df['Pickup Community Area']))|(np.isnan(df['Dropoff Community Area']))])) +  ' Trip Records with one trip end outside of Cook county out of ' + str(len(df)) )


            #select out the trips that have community area data and are missing census tract data 
            df['SUP_PRIVATE_TRIPS'] = np.where(df['Shared Trip Authorized'] == False, 1, 0)
            df['SUP_SHARED_TRIPS'] = np.where(df['Shared Trip Authorized'] == True, 1, 0)
            df = df[(np.isnan(df['Pickup Census Tract']))|(np.isnan(df['Dropoff Census Tract']))]
            df = df[~np.isnan(df['Pickup Community Area'])&(~np.isnan(df['Dropoff Community Area']))]


            df = df.groupby(by = ['Pickup Community Area', 'Dropoff Community Area', 'YEAR','MONTH','DAY'], as_index = False).sum()
            df = df.groupby(by = ['Pickup Community Area', 'Dropoff Community Area','YEAR','MONTH'], as_index = False).mean()

            print('There are ' +str(df.SUP_PRIVATE_TRIPS.sum() + df.SUP_SHARED_TRIPS.sum()) + ' Average Weekday Trips that are Suppressed!')

            #make a column to iterate through
            df['OD_PAIRS'] = df['Pickup Community Area'].astype(str) + '_' + df['Dropoff Community Area'].astype(str)

            #iterate through each of the suppressed Community Areas
            print('Working on assigning suppressed trips!')
            for od in df['OD_PAIRS'].unique():

                #select out the trips that are originating from the given community area
                od_trips = df[df['OD_PAIRS'] == od]

                #conflate the community area to the census tract centroids that fall within it
                df2 = od_trips[['SUP_PRIVATE_TRIPS', 'SUP_SHARED_TRIPS', 'Pickup Community Area', 'Dropoff Community Area']].merge(conflation[['GEOID','COMMUNITY_AREA']], how = 'left', left_on = 'Pickup Community Area' , right_on = 'COMMUNITY_AREA')
                df2 = df2.merge(conflation[['GEOID','area_num_1']], how = 'left', left_on = 'Dropoff Community Area' , right_on = 'area_num_1', suffixes = ('_PICKUP','_DROPOFF'))

                df2['SCALAR'] = np.random.dirichlet(np.ones(len(df2)))
                df2['SCALED_SUP_PRIVATE_TRIPS'] = df2['SUP_PRIVATE_TRIPS']*df2['SCALAR']
                df2['SCALED_SUP_SHARED_TRIPS'] = df2['SUP_SHARED_TRIPS']*df2['SCALAR']
                df3 = df3.append(df2)
            df3['TOD'] = tod       
            
            df4 = df3.append(df3)
        df4['MONTH'] = month
        print(str(df4.SCALED_SUP_PRIVATE_TRIPS.sum()))
        print(str(df4.SCALED_SUP_SHARED_TRIPS.sum()))
        
        df5 = df4.append(df4)
    df5['YEAR'] = year    
       
    df6 = df5.append(df5)
    

grouped = df6[['SCALED_SUP_PRIVATE_TRIPS','SCALED_SUP_SHARED_TRIPS', 'GEOID_PICKUP','GEOID_DROPOFF', 'MONTH','YEAR','Pickup Community Area', 'Dropoff Community Area']].groupby(by = ['GEOID_PICKUP', 'GEOID_DROPOFF','MONTH','YEAR'], as_index = False).agg(agg)        




print('Everyting is Complete!')

Working on year 2018
Working on month 11
Working on tod 1
Filtering the Data!
563808 Trip Records with Suppressed Origin and Destination out of 1800773
138037 Trip Records Outside of Chicago but within cook county out of 1800773
978219 Trip Records with both trip ends within Chicago out of 1800773
258746 Trip Records with one trip end outside of Cook county out of 1800773
There are 31928.0321587769 Average Weekday Trips that are Suppressed!
Working on assigning suppressed trips!
Working on tod 2
Filtering the Data!
330100 Trip Records with Suppressed Origin and Destination out of 1558829
89564 Trip Records Outside of Chicago but within cook county out of 1558829
1035235 Trip Records with both trip ends within Chicago out of 1558829
193494 Trip Records with one trip end outside of Cook county out of 1558829
There are 20861.294404417393 Average Weekday Trips that are Suppressed!
Working on assigning suppressed trips!
Working on tod 3
Filtering the Data!
835235 Trip Records with Suppresse

## Total Number of Suppressed Trips

In [ ]:
df_all.SCALED_SUP_PRIVATE_TRIPS.sum() + df_all.SCALED_SUP_SHARED_TRIPS.sum()

## Aggregate the data to an average weekday by TOD

In [12]:
grouped.to_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Monthly Suppressed Trips.csv')

## Visualize the Suppressed Trips

In [13]:
grouped['TOTAL_SUP_TRIPS'] = grouped.SCALED_SUP_PRIVATE_TRIPS + grouped.SCALED_SUP_SHARED_TRIPS

In [14]:
origin = grouped[['GEOID_PICKUP', 'TOTAL_SUP_TRIPS']].groupby(by = 'GEOID_PICKUP', as_index = False).sum()
dest = grouped[['GEOID_DROPOFF', 'TOTAL_SUP_TRIPS']].groupby(by = 'GEOID_DROPOFF', as_index = False).sum()

In [15]:
geo = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [16]:
origin[origin['GEOID_PICKUP'] == 17031081403]

,GEOID_PICKUP,TOTAL_SUP_TRIPS
137,17031081403,116.343629


In [17]:
origin['GEOID_PICKUP'] = origin.GEOID_PICKUP.astype(float)
dest['GEOID_DROPOFF'] = dest.GEOID_DROPOFF.astype(float)

In [18]:
geo['geoid10'] = geo.geoid10.astype(float)

In [19]:
centroids = pd.read_csv('C:/Workspace/TNC-Demand-Model/otp/points.csv')
centroids = gp.GeoDataFrame(centroids)

In [20]:
geo[geo['geoid10'] == 17031081403]

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,geometry
165,8,8.0,031,1.703108e+10,814.03,Census Tract 814.03,Small unpopulated area in CA 32,17,081403,"POLYGON ((-87.60953 41.89096, -87.60484 41.891..."


In [21]:
m = folium.Map([41.8781, -87.6298], zoom_start=11)
    
    
# Add the color for the chloropleth:
folium.Choropleth(
 geo_data=geo,
 name= "Suppressed Pickups",
 data=origin,
 columns = ['GEOID_PICKUP', 'TOTAL_SUP_TRIPS'],
 key_on='feature.properties.geoid10',
 fill_color='BuGn',
 fill_opacity=0.6,
 line_opacity=0.2,
 legend_name='Average Weekday Pickups',
 highlight = True
).add_to(m)

folium.Choropleth(
 geo_data=geo,
 name= "Suppressed Dropoffs",
 data=dest,
 columns = ['GEOID_DROPOFF', 'TOTAL_SUP_TRIPS'],
 key_on='feature.properties.geoid10',
 fill_color='BuGn',
 fill_opacity=0.6,
 line_opacity=0.2,
 legend_name='Average Weekday Dropoffs',
 highlight = True
).add_to(m)

feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)


for tract2 in centroids.GEOID:
    row = centroids[centroids['GEOID'] == tract2]
    folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)

m.add_child(feature_group)


folium.LayerControl().add_to(m)

m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/Suppressed Ridehailing Maps/Suppressed_Trips.html')

print('Everything is Complete!')

Everything is Complete!
